<a href="https://colab.research.google.com/github/lmcanavals/pcd/blob/master/1201_colabpotato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang-go

In [ ]:
%%file potatostart.go
package main

import (
	"fmt"
	"math/rand"
	"net"
	"os"
	"time"
)

var remote string

func main() {
	rand.Seed(time.Now().UnixNano())
	if len(os.Args) < 2 {
		return
	}
	remote = os.Args[1]
	send(rand.Intn(1000))
}

func send(num int) {
	cn, _ := net.Dial("tcp", remote)
	defer cn.Close()
	fmt.Fprintf(cn, "%d\n", num)
}


Writing potatostart.go


In [ ]:
%%file randpotato.go
package main

import (
	"bufio"
	"fmt"
	"math/rand"
	"net"
	"os"
	"strconv"
	"strings"
	"time"
)

var remote []string

func main() {
	rand.Seed(time.Now().UnixNano())
	if len(os.Args) < 3 {
        fmt.Println(":(")
		return
	}
	host := os.Args[1]
	remote = os.Args[2:]

	ln, _ := net.Listen("tcp", host)
	defer ln.Close()

	fmt.Printf("Listening on %s...\n", host)

	for {
		cn, _ := ln.Accept()
		go handle(cn)
	}
}

func handle(cn net.Conn) {
	defer cn.Close()
	r := bufio.NewReader(cn)
	strNum, _ := r.ReadString('\n')
	num, _ := strconv.Atoi(strings.TrimSpace(strNum))
	fmt.Printf("Nro: %s", strNum)
	if num == 0 {
		fmt.Println("He perdido...")
	} else {
		send(num - 1)
	}
}

func send(num int) {
	selected := remote[rand.Intn(len(remote))]
	cn, _ := net.Dial("tcp", selected)
	fmt.Printf("Sending %d to %s...\n", num, selected)
	defer cn.Close()
	fmt.Fprintf(cn, "%d\n", num)
}


Overwriting randpotato.go


In [ ]:
!go build randpotato.go

In [ ]:
%%bash

./randpotato localhost:8000 localhost:8001 localhost:8002 localhost:8003 localhost:8004 &
sleep 1
./randpotato localhost:8001 localhost:8000 localhost:8002 localhost:8003 localhost:8004 &
sleep 1
./randpotato localhost:8002 localhost:8001 localhost:8000 localhost:8003 localhost:8004 &
sleep 1
./randpotato localhost:8003 localhost:8001 localhost:8002 localhost:8000 localhost:8004 &
sleep 1
./randpotato localhost:8004 localhost:8001 localhost:8002 localhost:8003 localhost:8000 &
sleep 1

go run potatostart.go localhost:8000

sleep 5
pkill randpotato

In [ ]:
!ps ax

In [ ]:
%%file sp.go
package main

import (
    "fmt"
    "time"
    "os"
)

func main() {
    time.Sleep(time.Second)
    for i := 0; i < 100; i++ {
        fmt.Printf("%s %d\n", os.Args[1], i)
        time.Sleep(time.Millisecond * 10)
    }
}

Overwriting sp.go


In [ ]:
%%bash
go build sp.go

./sp "procesoA" &
sleep 1
./sp "  procesoB" &